Shreya Akurathi 
200968188
Week 4 


In [ ]:
!pip install easyAI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 2.3 MB/s eta 0:00:00


In [ ]:
from easyAI import TwoPlayerGame
from easyAI.Player import Human_Player

In [ ]:
class TicTacToe(TwoPlayerGame):
    """The board positions are numbered as follows:
    1 2 3
    4 5 6
    7 8 9
    """

    def __init__(self, players):
        self.players = players
        self.board = [0 for i in range(9)]
        self.current_player = 1  # player 1 starts.

    def possible_moves(self):
        return [i + 1 for i, e in enumerate(self.board) if e == 0]

    def make_move(self, move):
        self.board[int(move) - 1] = self.current_player

    def unmake_move(self, move):  # optional method (speeds up the AI)
        self.board[int(move) - 1] = 0

    def lose(self):
        """ Has the opponent "three in line ?" """
        return any(
            [
                all([(self.board[c - 1] == self.opponent_index) for c in line])
                for line in [
                    [1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9],  # horiz.
                    [1, 4, 7],
                    [2, 5, 8],
                    [3, 6, 9],  # vertical
                    [1, 5, 9],
                    [3, 5, 7],
                ]
            ]
        )  # diagonal

    def is_over(self):
        return (self.possible_moves() == []) or self.lose()

    def show(self):
        print(
            "\n"
            + "\n".join(
                [
                    " ".join([[".", "O", "X"][self.board[3 * j + i]] for i in range(3)])
                    for j in range(3)
                ]
            )
        )

    def scoring(self):
        return -100 if self.lose() else 0




In [ ]:
if __name__ == "__main__":

    from easyAI import AI_Player, Negamax

    ai_algo = Negamax(6)
    TicTacToe([Human_Player(), AI_Player(ai_algo)]).play()


. . .
. . .
. . .

Player 1 what do you play ? 5

Move #1: player 1 plays 5 :

. . .
. O .
. . .

Move #2: player 2 plays 1 :

X . .
. O .
. . .

Player 1 what do you play ? 9

Move #3: player 1 plays 9 :

X . .
. O .
. . O

Move #4: player 2 plays 3 :

X . X
. O .
. . O

Player 1 what do you play ? 2

Move #5: player 1 plays 2 :

X O X
. O .
. . O

Move #6: player 2 plays 8 :

X O X
. O .
. X O

Player 1 what do you play ? 7

Move #7: player 1 plays 7 :

X O X
. O .
O X O

Move #8: player 2 plays 4 :

X O X
X O .
O X O

Player 1 what do you play ? 6

Move #9: player 1 plays 6 :

X O X
X O O
O X O


In [ ]:
import easyAI

In [ ]:
dir(easyAI)

['AI',
 'AI_Player',
 'DUAL',
 'DictTranspositionTable',
 'HashTranspositionTable',
 'Human_Player',
 'Negamax',
 'NonRecursiveNegamax',
 'Player',
 'SSS',
 'TranspositionTable',
 'TwoPlayerGame',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'mtd',
 'solve_with_depth_first_search',
 'solve_with_iterative_deepening']

# **Iterative Deepening**

Use iterative deepening when we do not know the depth of our solution and have to search a very large state space.

In [ ]:
from easyAI.AI import Negamax
from easyAI.AI import TranspositionTable
from easyAI.Player import AI_Player

def id_solve(game, ai_depths, win_score, scoring=None,
          tt=None, verbose=True):
    """
    Solves a game using iterative deepening, i.e. determines if by playing
    perfectly the first player can force a win, or whether it will always
    lose against a perfect opponent.
    
    
    This algorithm explores the game by using several times the Negamax
    algorithm, always starting at the initial state of the game, but
    taking increasing depth (in the list ai_depths) until the score of
    the initial condition indicates that the first player will certainly
    win or loose, in which case it stops.
    The use of transposition table leads to speed gain as the results
    of shallower searches are used to help exploring the deeper ones.
        
    Parameters
    -----------
    
    ai_depths:
      List of AI depths to try (e.g. [5,6,7,8,9,10])
      
      
    win_score:
      Score above which a score means a win.
    
    scoring:
      Scoring function (see doc of class Negamax)
    
    tt:
      An optional transposition table to speed up computations.
    
    verbose:
      If set to ``True``, will print a summary of the best move
      after each depth tried.
        
    Returns
    --------
    
    (result, depth, move):
      As below
    
    result: 
      Either 1 (certain victory of the first player) or -1
      (certain defeat) or 0 (either draw, or the search was not
      deep enough)
      
    depth:
      The minimal number of moves before victory (or defeat)
    
    move:
      Best move to play for the first player.

    Also returns with ``tt`` set.
      Will be None if ``use_tt`` was set to false, else will be a
      transposition table containing all the relevant situations to play
      a perfect game and can be used with ``AI_player(tt)``
      
    """
    
    if not hasattr(game, 'players'): # the user provided a Game class
        game = game(players = [AI_Player(None), AI_Player(None)])
    
    for depth in ai_depths:
        ai = Negamax(depth, scoring, tt= tt)
        ai(game)
        alpha = ai.alpha
        if verbose:
             print( "d:%d, a:%d, m:%s"%(depth, alpha, str(game.ai_move)))
        if abs(alpha) >= win_score:
            break
    
    # 1:win, 0:draw, -1:defeat
    result = (+1 if alpha>= win_score else (
             -1 if alpha <= -win_score else 0))
    
    return result, depth, game.ai_move

id_solve solves a game using iterative deepening: it explores the game by using several times the Negamax algorithm, always starting at the initial state of the game, but taking increasing depth (in the list ai_depths) until the score of the initial condition indicates that the first player will certainly win or loose, at which case it stops

In [ ]:
r,d,m = id_solve( TicTacToe, ai_depths=range(2,20), win_score=100)

d:2, a:0, m:1
d:3, a:0, m:1
d:4, a:0, m:1
d:5, a:0, m:1
d:6, a:0, m:1
d:7, a:0, m:1
d:8, a:0, m:1
d:9, a:0, m:1
d:10, a:0, m:1
d:11, a:0, m:1
d:12, a:0, m:1
d:13, a:0, m:1
d:14, a:0, m:1
d:15, a:0, m:1
d:16, a:0, m:1
d:17, a:0, m:1
d:18, a:0, m:1
d:19, a:0, m:1


*  We obtain **a=0**, meaning that if both players play perfectly, the game will always** end in a draw** (-1 would have meant always lose, 1 meaning always winning
)
* **d=19**, which means that **the wins will be in nineteen moves ** (i.e. around 8 moves per player) or less
* **m=1,** which indicates that the **first player’s first move should be 1.**








# **Depth First Search**

In [ ]:
def df_solve(game, win_score, maxdepth=50, tt=None, depth=0):
    """ 
    Solves a game using a depth-first search: the game is explored until
    endgames are reached.
    
    The endgames are evaluated to see if there are victories or defeats.
    Then, a situation in which every move leads to a defeat is labelled
    as a (certain) defeat, and a situation in which one move leads to a
    (certain) defeat of the opponent is labelled as a (certain) victory.
    Situations are evaluated until the initial condition receives a label
    (victory or defeat). Draws are also possible.
    
    This algorithm can be faster but less informative than ``id_solve``,
    as it does not provide 'optimal' strategies (like shortest path to
    the victory). It returns simply 1, 0, or -1 to indicate certain
    victory, draw, or defeat of the first player.
        
    Parameters
    -----------
    
    game:
      An Game instance, initialized and ready to be played.
      
    win_score:
      Score above which a score means a win.
    
    maxdepth:
      Maximal recursion depth allowed.
    
    tt:
      An optional transposition table to speed up computations.
      
    
    depth:
      Index of the current depth (don't touch that).
      
    Returns
    --------
    
    result
      Either 1 (certain victory of the first player) or -1
      (certain defeat) or 0 (either draw, or the search was not
      deep enough)
    
    """
    
    
    # Is there a transposition table and is this game in it ?
    lookup = None if (tt is None) else tt.lookup(game)
    if lookup != None:
        return lookup['value']
        
    if (depth == maxdepth):
        raise "Max recursion depth reached :("
    
    if game.is_over():
        score = game.scoring()
        value = 1 if (score>=win_score) else (-1 if -score>=win_score else 0)
        #tt.store(game=game, value=value, move=None)
        return  value
    
    possible_moves = game.possible_moves()
    
    state = game
    unmake_move = hasattr(state, 'unmake_move')
    
    best_value, best_move = -1, None
    
    for move in possible_moves:
        
        if not unmake_move:
            game = state.copy() # re-initialize move
        
        game.make_move(move)
        game.switch_player()
        
        move_value = - df_solve(game,  win_score,  maxdepth, tt, depth+1)
        
        if unmake_move:
            game.switch_player()
            game.unmake_move(move)
        
        if move_value == 1:
            #tt.store(game=state, value=1, move=move)
            return move_value
        
        if move_value == 0 and best_value==-1:
            # Is forcing a draw possible ?
            best_value = 0
            best_move = move
    
    #tt.store(game=state, value=best_value, move=best_move)
    
    return best_value

In [ ]:
game = TicTacToe(players = [Human_Player(), AI_Player(ai_algo)]) # the players are not important

r = df_solve(game, win_score= 90)

The game is explored until endgames are reached and these endgames are evaluated to see if their are victories or defeats (or draws). Then, a situation in which every move leads to a defeat is labelled as a (certain) defeat, and a situation in which one move leads to a (certain) defeat of the opponent is labelled as a (certain) victory. This way we come back up to the root (initial condition) which receives a label, which is returned.

In [ ]:
r

0

**We get 0 which means the game will end in a draw.**
r is either -1 (certain defeat of the first player against a perfect opponent), 0 (it is possible to force a draw, but not to win), or 1 (certain victory if the first player plays perfectly).

the iterative deepening function gives more information about the posssible gameplays than depth first search.

**Iterative deepening:**
This algorithm starts with a depth limit of 1 and keeps increasing the depth limit until a solution is found. It performs depth-first search at each depth limit.

**Depth-first search:** This algorithm starts at the root node and explores each branch as far as possible before backtracking.